In [ ]:
## Importing Necessary Libraries
import requests
import json
import urllib3
import pandas as pd
from datetime import datetime
urllib3.disable_warnings()
import calendar
import warnings
import numpy as np
import os

writer = pd.ExcelWriter('C:\\Users\\oazed\\OneDrive\\Desktop\\VOLUME\\202112- ALL VOLUMES ISP+GCEX 1 to 13.xlsx', engine='xlsxwriter')

## Getting Terminus Data Using ISPrime API
API_URL_BASE = 'https://api.isprimefx.com/api/'
USERNAME = 'operations@alphatrade.com.au'
PASSWORD = 'promptshadeimpose'



# login and extract the authentication token
response = requests.post(API_URL_BASE + 'login', 
headers={'Content-Type': 'application/json'},
data=json.dumps({'username': USERNAME, 'password': PASSWORD}), verify=False)

token = response.json()['token']
headers = {'X-Token': token}
#print(token)

 ## Data Function to get rid of Sat & Sun days in each week 
def findDay(date):
    dday = datetime.strptime(date, '%Y-%m-%d').weekday()
    return (calendar.day_name[dday])
  
## Getting SPOT Trades for required day/s 
data_1 = pd.DataFrame()
arr_1 = []

############## PLEASE CHANGE THE DATES HERE CAREFULLY
for x in range(1,14):  ### CHANGE THE RANGE
    params = {'date': '2021-12-'+ str(x), ### CHANGE THE DATE MONTH
    'tradeType':'Spot',
    'category':'FX'}
    
    y = '2021-12-'+str(int(x))   ### CHANGE THE DATE SIMILAR TO THE PARAMS
    if (findDay(y) == 'Saturday') or (findDay(y) == 'Sunday'):
        continue
    
    response = requests.get(API_URL_BASE + 'trades', headers=headers, params=params, verify=False)
    d = json.dumps(response.json(), indent=4)
    df = pd.read_json(d)
    arr_1.append(df)
   
for i in range(0, len(arr_1)):
    data_1 = pd.concat([data_1, arr_1[i]], ignore_index=True, sort=False)
    



############### PLEASE CHANGE THE DATES HERE CAREFULLY

df_1 = pd.DataFrame()
lis_1 = []
for x in range(1,14):   ### CHANGE THE RANGE
    params = {'date': '2021-12-'+ str(x), ### CHANGE THE DATE MONTH
    'tradeType':'Spot', 
    'category':'IndexSwap'}
    
    y = '2021-12-'+str(int(x))    ### CHANGE THE DATE SIMILAR TO THE PARAMS
    if (findDay(y) == 'Saturday') or (findDay(y) == 'Sunday'):
        continue
    
    response = requests.get(API_URL_BASE + 'trades', headers=headers, params=params, verify=False)
    d = json.dumps(response.json(), indent=4)
    df = pd.read_json(d)
    lis_1.append(df)
   
for i in range(0, len(lis_1)):
    df_1 = pd.concat([df_1, lis_1[i]], ignore_index=True, sort=False)

    
################ PLEASE CHANGE THE DATE HERE CAREFULLY
other_1 = pd.DataFrame()
marks_1 = []

for x in range(1,14):  ### FOLLOW THE ABOVE RANGE
    params = {'date': '2021-12-'+ str(x)} ### FOLLOW ABOVE
    y = '2021-12-'+str(int(x))  ### FOLLOW ABOVE

    if (findDay(y) == 'Saturday') or (findDay(y) == 'Sunday'):
        continue
    
    response = requests.get(API_URL_BASE + 'marks', headers=headers, params=params, verify=False)
    e = json.dumps(response.json(), indent=4)
    df3 = pd.read_json(e)
    marks_1.append(df3)

for i in range(0, len(marks_1)):
    other_1 = pd.concat([other_1, marks_1[i]], ignore_index=True, sort=False)
    
other = other_1
other = other.rename(columns={'price':'eodPrice'})
other.date = pd.to_datetime(other.date)


## Merged Spot and IndexSwap

merged = pd.concat([data_1, df_1])
mother = merged[['tradeType', 'category', 'clientReference', 'accountNo', 'tradeDate', 'valueDate', 'tradeDttm', 'instrumentName', 'instrument', 'tradedCurrency', 'tradedQuantity', 'baseQuantity', 'contraQuantity','price', 'side', 'cancelled']]
mother.tradeDate = pd.to_datetime(mother.tradeDate)
'''Taking the absolute value of the Base value and Contract value '''

mother['Base Absolute Amount']= mother['tradedQuantity'].abs()
mother['Contra Absolute Amount']= mother['contraQuantity'].abs()

### Dictionary for getting CFD's origianl value using original multipliers
instrument_dict = { 'IDX':{
                            'US': 10,
                            'DE':10,
                            'JP':100,
                            'UK':10,
                            'AU':10,
                            'HK':10,
                            'HKD':10},
                    'SPT':{'CO':10}
                  }

#### Function for getting the Origianl Contract value 
def cust_contract(x):
    placeholder = 0
    placebo = 0
    if '.' in x[0]:
        cols = x[0].split('.')
        if cols[0].startswith('SPT'):
            placebo+= x[1] * x[2] * instrument_dict[cols[0]][cols[1]]
            return placebo
        else:
            placeholder += x[1] * (x[2]/ instrument_dict[cols[0]][cols[1]])
            return placeholder
    else:
        return x[3]
mother['Contract'] = mother[['instrument', 'price','Base Absolute Amount', 'Contra Absolute Amount']].apply(cust_contract, axis = 1) 

### CountryDict dictionary will be used to get the eod for the corresponding instruments
countryDict = {
        "IDX.AU": "AUD/USD",
        "IDX.EU": "EUR/USD",
        "IDX.FR": "EUR/USD",
        "IDX.DE": "EUR/USD",
        "IDX.ES": "EUR/USD",
        "IDX.HK": "HKD/USD",
        "IDX.JP": "JPY/USD",
        "IDX.UK": "GBP/USD",
        "IDX.US": "USD/USD",
        "CO": "USD/USD",
        "BTC": "USD/USD"
}

mother.rename(columns={'tradeDate':'date'}, inplace=True)

mother.set_index(['date','instrument'], inplace=True)

other.drop(['category', 'instrumentName'], axis=1, inplace=True)

conv = dict()
''' findUSD function and applyUSD will create another dictionary calls conv which will have multi indexed EOD entries for each date
and corrsponding EOD instrumnents for each date Also findUSD will find Any instrument/USD value and put those in 
conv dictionary for example if AUD/CAD is there it will find AUD/USD if not found then it will do 1/(USD/AUD) and
put the value in conv dictionary '''

def findUSD(s):
    date = s['date']
    ins = s['instrument']
    rate = s['eodPrice']
    
    x = ins.split('/')

    if date not in conv:
        conv[date] = dict()
        conv[date]['USD/USD'] = 1.0
    
    if len(x) != 2:
        return
        
    if x[1] == 'USD':
        conv[date][ins] = rate
        
    if x[0] == 'USD':
        ins = x[1] + '/' + x[0]
        conv[date][ins] = 1/rate

def applyUSD(s):
    date = s['date']
    ins = s['instrument']
    rate = s['eodPrice']
   
    x = ins.split('/')
    
    if len(x) != 2:
        for kwd in countryDict.keys():
            if kwd in ins:
                if countryDict[kwd] in conv[date]:
                    return conv[date][countryDict[kwd]]
        
    if 'USD' in x:
        if x[0] == 'USD':
            return 1/rate
        elif x[1] == 'USD':
            return rate
    
    ins0 = x[0]+'/'+'USD'
    ins1 = x[1]+'/'+'USD'
    
    if ins0 in conv[date]:
        return conv[date][ins0]
    
    if ins1 in conv[date]:
        return rate * conv[date][ins1]
        
other.apply(findUSD, axis=1)

other['IN_USD'] = other.apply(applyUSD, axis=1)

other.set_index(['date','instrument'], inplace=True)
trade = mother.join(other, how='inner')
trade= trade.reset_index()

#### This function will get overall volumes in USD Base abs amount/Contract amount * USD(eod) Values
def math_calc(x):
    placeholder = 0
    if '/' in x[0]:
        cols = x[0].split('/')
        if cols[1] =='USD':
            placeholder+= x[1]* x[4]
            return placeholder
        elif cols[0] == 'USD':
            placeholder+= x[1]*1
            return placeholder
        else:
            placeholder+= x[1]* x[4]
            return placeholder
    if '.' in x[0]:
        placeholder+= x[2] * x[4]
        return placeholder

trade['All_USD'] = trade[['instrument','Base Absolute Amount','Contract','eodPrice','IN_USD']].apply(math_calc,axis=1)
trade.to_excel(writer, sheet_name='Sheet1-ISP Trades')

### Now all trade from ISPrime is grouped by account No and category aggregated by Sum
piv = trade.groupby(["accountNo","category"]).agg({"All_USD":"sum"}).reset_index()

cryptos =['BTCUSD', 'BCHUSD', 'EOSUSD','ETHUSD','LTCUSD', 'XRPUSD','LNKUSD','USTUSD']
cfds = [ 'US500','NAS100','US30' ,'AUS200','EUSTX50','HK50','JPN225','UK100' ,'GER30','CN50' ,'FRA40']


#### Dictionary for calculating House, Cost & Net Commission for FX, CFDs 
calc_dict = {
    'APT701': {"FX": {
            "COST": 12,
            "HOUSE": 7}},   
    
    'APT001': {"FX": {
            "COST": 10,
            "HOUSE": 7}},    
    
    'APT702': {"FX": {
            "COST": 10,
            "HOUSE": 7}},
     'APT703': {"FX": {
            "COST": 10,
            "HOUSE": 7}},
     'APT704': {"FX": {
            "COST": 12,
            "HOUSE": 7}},
     'APT705': {"FX": {
            "COST": 10,
            "HOUSE": 7}},
    'APT411': {"IndexSwap": {
            "COST": 15,
            "HOUSE": 8}}, 
    'APT412': {"IndexSwap": {
            "COST": 10,
            "HOUSE": 8}},
    'APT413': {"IndexSwap": {
            "COST": 10,
            "HOUSE": 8}},
    'APT414': {"IndexSwap": {
            "COST": 12,
            "HOUSE": 8}},
     'APT415': {"IndexSwap": {
            "COST": 10,
            "HOUSE": 8}},
}

def target_function(features, comms):
    
    _comms = comms
    
    _t_val = 0
    _accName = features[0]
    _type = features[1]
    _value = features[2]
    
    _t_val = _value * calc_dict[_accName][_type][_comms] / 10**6
    
    return _t_val

### APPLYING FUNCTION
house = piv.apply(lambda x: target_function(x, "HOUSE"), axis=1)
cost = piv.apply(lambda x: target_function(x, "COST"), axis=1)
piv['Comms(Cost$M)'] = cost
piv['Comms(House$M)']=house
piv["Net Comms($M)"] = piv['Comms(Cost$M)'] - piv['Comms(House$M)']
piv['All_USD'] = piv['All_USD']/10**6
piv_table = pd.pivot_table(piv, values='All_USD', index=[ 'accountNo','Comms(Cost$M)','Comms(House$M)','Net Comms($M)'],columns=['category']).reset_index()
piv_table = piv_table.rename(columns={'IndexSwap': 'CFDS'})
try:
    piv_table = piv_table[['accountNo', 'FX', 'CFDS','Comms(Cost$M)', 'Comms(House$M)', 'Net Comms($M)']]
except KeyError:
    piv_table = piv_table[['accountNo', 'FX', 'Comms(Cost$M)', 'Comms(House$M)', 'Net Comms($M)']]
piv_table.to_excel(writer, sheet_name='Sheet2-ISP Summary')


######## GETTING GCEX VOLS
## Getting the statements
cwd = os.path.abspath(r'C:\Users\oazed\AlphaTrade Dropbox\Alpha Trade Pty Limited\Operation\Statements\GCEX\Daily Statements')
files = os.listdir(cwd)
## Getting the eods   **** ADD THE EOD PRICES CSV SHEET HERE
eod = pd.read_csv("C:\\Users\\oazed\\Downloads\\2021-12-15 014533 - Hub Symbol EOD Price History - default.csv")
eod = eod[['Timestamp','Symbol', 'Bid Price']]
eod = eod.rename(columns={'Timestamp':'date'})
eod["date"] = pd.to_datetime(eod["date"])
eod['date'] = eod['date'].dt.date
eod = eod.rename({'Symbol':'instrument'},axis=1)
### EOD dictionary for translate
eods_dict= {
    'US500' : 'SPM/USD',
    'NAS100': 'NDM/USD',
    'US30' : 'DJI/USD',
    'AUS200': 'AXI/AUD',
    'EUSTX50':'STX/EUR',
    'HK50': 'HSI/HKD',
    'JPN225': 'JPU/JPY',
    'UK100' : 'UKI/GBP',
    'GER30': 'GEM/EUR',
    'CN50' : 'CNI/USD',
    'FRA40':'FRI/EUR',
    'WTIUSD':'WTI/USD'
}
 
#### Function that translate
def abar_banao(x):
    ins = x['instrument']
    if ins in eods_dict.keys():
        return eods_dict[ins]
    else:
        return ins
    
vals = eod.apply(abar_banao, axis=1)
eod['instrument'] = vals

#### ALL the CFDS , CRYPTOs, and FX Vals

strathos_714=[]
lis3= []

excel_files = [x for x in os.listdir(cwd) if x.endswith(".xls")]

  
    
for file in excel_files:
    if file.startswith('714'):
#         if '2021-09-12' in file:  ## BREAK IT OR UNTIL U WANT
#             break
        lis3.append(file)
                    
for i in range(lis3.index("714_2021-12-01_2021-12-01.xls"),len(lis3)):  ## CHANGE THE DATE & MONTH HERE
    excel_path =os.path.join(cwd,lis3[i])
    this_df = pd.read_excel(excel_path)
    this_df = this_df.rename(columns={'GC Exchange Limited': 'col1'})
    this_df['filename'] = lis3[i]
    try:
        this_df = this_df.iloc[this_df.col1[this_df.col1 == 'Executed Exchange Trades'].index.tolist()[0]+1:this_df.col1[this_df.col1 == 'Closed Exchange Trades'].index.tolist()[0]+1]
    except IndexError:
        this_df = this_df.iloc[this_df.col1[this_df.col1 == 'Executed CFD Trades'].index.tolist()[0]+1:this_df.col1[this_df.col1 == 'Closed CFD Trades'].index.tolist()[0]+1]
    strathos_714.append(this_df)
    strath = pd.concat(strathos_714)
    


## FOR FX


strathos_fx=[]
lis_stra=[] 
    
    

for file in excel_files:
    if file.startswith('714'):
#         if '2021-09-12' in file:  ## BREAK IT OR UNTIL U WANT
#             break
        lis_stra.append(file)             
for i in range(lis_stra.index("714_2021-12-01_2021-12-01.xls"),len(lis_stra)):  ## CHANGE THE DATE & MONTH HERE
    excel_path =os.path.join(cwd,lis_stra[i])
    this_df = pd.read_excel(excel_path)
    this_df = this_df.rename(columns={'GC Exchange Limited': 'col1'})
    this_df['filename'] = lis_stra[i]
    try:
        this_df = this_df.iloc[this_df.col1[this_df.col1 == 'Executed Trades'].index.tolist()[0]+1:this_df.col1[this_df.col1 == 'Executed Exchange Trades'].index.tolist()[0]+1]
    except IndexError:
        this_df = this_df.iloc[this_df.col1[this_df.col1 == 'Executed Trades'].index.tolist()[0]+1:this_df.col1[this_df.col1 == 'Executed CFD Trades'].index.tolist()[0]+1]
    strathos_fx.append(this_df)
    stra = pd.concat(strathos_fx)
    


trades = pd.concat([strath])
trades = trades.drop(['Unnamed: 10'], axis=1)
trades = trades.dropna().reset_index()
df_tr = trades[trades['col1'].apply(lambda x: "Trade Date" in x)]
trades = trades.drop(df_tr.index)
trades = trades.drop(['index'], axis=1)
trades = trades[['col1', 'Unnamed: 3','Unnamed: 4','Unnamed: 7','Unnamed: 8','Unnamed: 9','filename' ]]
trades = trades.rename(columns={'col1': 'Date', 'Unnamed: 3':'Execution ID', 'Unnamed: 4':'Instrument', 'Unnamed: 7':'Rate', 'Unnamed: 8':'Dealt Qty', 'Unnamed: 9':'Settle CCY', 'filename':'AccName'})
trades.AccName = trades.AccName.str[0:3]
trades['Rate'].replace(to_replace=",", value="", regex=True, inplace=True)
trades['Dealt Qty'].replace(to_replace=",", value="", regex=True, inplace=True)
trades[["Rate", "Dealt Qty"]] = trades[["Rate", "Dealt Qty"]].apply(pd.to_numeric)
trades.rename(columns={'Date':'date','Instrument':'instrument'}, inplace=True)
trades["date"] = pd.to_datetime(trades["date"])
tr_purge = trades[trades['Execution ID'].apply(lambda x: len(x)>19)]
trades = trades.drop(tr_purge.index).reset_index()
trades= trades.drop(['Execution ID'],axis=1)
def banao(x):
    bana = x['instrument']
    x = bana.split('/')
    return x[1]+'/'+x[2]
settle = trades.apply(banao,axis=1)
trades['instrument'] = settle


fx_trades = pd.concat([stra]) 
fx_trades = fx_trades.dropna()
fx_trades= fx_trades.reset_index()
fx_trades= fx_trades.drop(['index'], axis=1)
df_c = fx_trades[fx_trades['col1'].apply(lambda x: "Trade Date" in x)]
fx_trades = fx_trades.drop(df_c.index)
fx_trades = fx_trades[['col1', 'Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','filename' ]]
fx_trades = fx_trades.rename(columns={'col1': 'Date', 'Unnamed: 3':'ID','Unnamed: 4':'Instrument', 'Unnamed: 5':'Side', 'Unnamed: 6':'Dealt Amount', 'Unnamed: 7':'CCY One', 'Unnamed: 8':'Rate', 'Unnamed: 9':'Contra Qty', 'Unnamed: 10':'CCY Two', 'filename':'AccName'})
fx_trades= fx_trades.reset_index()
fx_trades= fx_trades.drop(['index'], axis=1)
fx_trades.AccName = fx_trades.AccName.str[0:3]
fx_trades["Contra Qty"].replace(to_replace=",", value="", regex=True, inplace=True)
fx_trades["Contra Qty"] = fx_trades["Contra Qty"].apply(pd.to_numeric)
fx_trades["Contra Qty"] = fx_trades["Contra Qty"].abs()
fx_trades["Dealt Amount"].replace(to_replace=",", value="", regex=True, inplace=True)
fx_trades["Dealt Amount"] = fx_trades["Dealt Amount"].apply(pd.to_numeric)
fx_trades["Dealt Amount"] = fx_trades["Dealt Amount"].abs()
fx_trades = fx_trades.rename({'Date':'date','Instrument':'instrument'},axis=1)
fx_trades["date"] = pd.to_datetime(fx_trades["date"])
fx_purge = fx_trades[fx_trades['ID'].apply(lambda x: len(x)>19)]
fx_trades = fx_trades.drop(fx_purge.index).reset_index()
fx_trades= fx_trades.drop(['ID','index'],axis=1)



other_dict= {
    'SPM/USD' : 'SPM/USD',
    'NDM/USD': 'NDM/USD',
    'DJI/USD' : 'DJI/USD',
    'AXI/AUD': 'AXI/AUD',
    'STX/EUR':'STX/EUR',
    'HSI/HKD': 'HSI/HKD',
    'JPU/JPY': 'JPU/JPY',
    'UKI/GB' : 'UKI/GBP',
    'GEM/EUR': 'GEM/EUR',
    'CNI/USD' : 'CNI/USD',
    'FRI/EUR':'FRI/EUR',
    'XBRUSD':'XBRUSD',
    'XTIUSD':'XTIUSD',
    'WTI/USD':'WTI/USD'
}

trades.set_index(['date','instrument'], inplace=True)

conv = dict()

def findUSD(s):
    date = s['date']
    ins = s['instrument']
    rate = s['Bid Price']
    
    if '/' not in ins:
        if ins.endswith('USD'):
            conv[date][ins] = rate
    x = ins.split('/')

    if date not in conv:
        conv[date] = dict()
        conv[date]['USD/USD'] = 1.0
    
    if len(x) != 2:
        return
        
    if x[1] == 'USD':
        conv[date][ins] = rate
        
    if x[0] == 'USD':
        ins = x[1] + '/' + x[0]
        conv[date][ins] = 1/rate

def applyUSD(s):
    date = s['date']
    ins = s['instrument']
    rate = s['Bid Price']
    
    x = ins.split('/')
    
    if len(x) != 2:
        for kwd in other_dict.keys():
            if kwd in ins:
                if other_dict[kwd] in conv[date]:
                    return conv[date][other_dict[kwd]]
     
    if 'USD' not in x:
        if x[0] == 'AXI':
            return conv[date]['AUD/USD'] * rate
        if x[0] == 'UKI':
            return conv[date]['GBP/USD'] * rate
        if x[0] == 'GEM':
            return  conv[date]['EUR/USD'] * rate
        if x[0] == 'CNI':
            return conv[date]['CNH/USD'] * rate
        if x[0] == 'HSI':
            return conv[date]['HKD/USD'] * rate
        if x[0] == 'JPU':
            return conv[date]['JPY/USD'] * rate
        if x[0] == 'FRI':
            return conv[date]['EUR/USD'] * rate
        
        
    if 'USD' in x:
        if x[0] == 'USD':
            return 1/rate
        elif x[1] == 'USD':
            return rate
    
    ins0 = x[0]+'/'+'USD'
#     ins1 = x[1]+'/'+'USD'
    
    if ins0 in conv[date]:
        return conv[date][ins0]
    
#     if ins1 in conv[date]:
#         return rate * conv[date][ins1]
        
eod.apply(findUSD, axis=1)

eod['IN_USD'] = eod.apply(applyUSD, axis=1)

eod.set_index(['date','instrument'], inplace=True)
trades = trades.join(eod, how='left')
trades= trades.reset_index()

_list = trades['Settle CCY'].unique().tolist()
### Handling CFD trades and transalting into USD VOLs

def eod_cfd(x):
    val1 = x['Settle CCY']
    val2 = x['Rate']
    val3 = x['Dealt Qty']
    val4 = x['IN_USD']
    for ins in _list:
        return val4 * val3

trades['Value']= trades.apply(eod_cfd,axis=1).abs()

fx_trades.set_index(['date','instrument'], inplace=True)

fx_trades = fx_trades.join(eod, how='left')
fx_trades= fx_trades.reset_index()
fx_trades = fx_trades.dropna()


fx_list = fx_trades['CCY Two'].unique().tolist()

#### Handling FX trades and translating into USD VOLs
def eod_fx(x):
    val1 = x['IN_USD']
    val2 = x['Dealt Amount']
    val3 = x['instrument']
    ins = val3.split('/')
    if ins[0] == 'USD':
        return val2
    else:
        return val1 * val2
try:        
    fx_trades['Value']= fx_trades.apply(eod_fx,axis=1).abs()
except:
    pass

trades = trades.drop(['Bid Price'],axis=1)

cryptos =['BTC', 'BCH', 'EOS','ETH','LTC', 'XRP','LNK','UST']


def custom_instrument(x):
    if "/" in x:
        vals = x.split("/")
        if vals[0] in cryptos:
            return "CRYPTOS"
        else:
            return "CFDS"
trades["Type"] = trades["instrument"].apply(lambda x: custom_instrument(x))

########## GCEX CFDS and CRYPTOS Dump
trades.to_excel(writer, sheet_name='Sheet3-GCEX(CFDS)')

piv = trades.groupby(["AccName","Type"]).agg({"Value":"sum"}).reset_index()


piv['Value'] = piv['Value']/10**6

fx_trades = fx_trades.drop(['Side','CCY One','Bid Price'],axis=1)

fx_trades['Type']='FX'

##### GCEX FX DUMP
fx_trades.to_excel(writer, sheet_name='Sheet4-GCEX(FX)')

try:
    piv_fx = fx_trades.groupby(["AccName","Type"]).agg({"Value":"sum"}).reset_index()
except:
    pass
    

    piv_fx['Value'] = piv_fx['Value']/10**6
except:
    pass
try:
    piv_table_gcx = pd.concat([piv, piv_fx])
except:
    piv_table_gcx = piv
piv_table_gcx = pd.pivot_table(piv_table_gcx, values='Value', index=[ 'AccName','Comms(Cost$M)','Comms(House$M)','Net Comms($M)'],columns=['Type']).reset_index()
try:
    piv_table_gcx = piv_table_gcx[['AccName', 'FX', 'CFDS','CRYPTOS','Comms(Cost$M)', 'Comms(House$M)', 'Net Comms($M)']]
except:
    piv_table_gcx = piv_table_gcx[['AccName', 'CRYPTOS','Comms(Cost$M)', 'Comms(House$M)', 'Net Comms($M)']]

piv_table_gcx = piv_table_gcx.rename(columns={'AccName':'accountNo'})

##### ONLY GCEX AGG DUMP
piv_table_gcx.to_excel(writer, sheet_name='Sheet5-GCEX Sum')

merged = pd.concat([piv_table, piv_table_gcx])
merged = merged.rename(columns={'accountNo': 'ClientName'})

clients = {
    'APT701':'FooFx - ISP',
    'APT702':'Azure - ISP',
    'APT703':'Strathos - ISP',
    'APT704': 'Realms - ISP',
    'APT705': 'Omnibus - ISP',
    'APT411':'FooFx - ISP',
    'APT412': 'Azure - ISP',
    'APT413': 'Strathos - ISP',
    'APT414':'Realms - ISP',
    'APT415':'Omnibus - ISP',
    '709':'FooFx - GCEX',
    '713':'Azure - GCEX',
    '714':'Strathos - GCEX',
    '715': 'Realms - GCEX'
}

def client_name(x):
    name =x['ClientName'] 
    return clients[name]

all_clients = merged.apply(client_name,axis=1)
merged['ClientName'] = all_clients
merged = merged.reset_index()
merged = merged.drop(['index'],axis=1)
try:
    merged = merged[['ClientName', 'FX','CFDS', 'CRYPTOS','Comms(Cost$M)', 'Comms(House$M)', 'Net Comms($M)']]
except:
    merged = merged[['ClientName', 'FX','CRYPTOS','Comms(Cost$M)', 'Comms(House$M)', 'Net Comms($M)']]
try:
    merged_piv = merged.append(merged[['FX', 'CFDS', 'CRYPTOS','Comms(Cost$M)', 'Comms(House$M)', 'Net Comms($M)']].sum().rename('Grand Total')).fillna('')
except:
    merged_piv = merged.append(merged[[ 'FX','CRYPTOS','Comms(Cost$M)', 'Comms(House$M)', 'Net Comms($M)']].sum().rename('Grand Total')).fillna('')

#### BOTH ISP+GCEX AGGREGATION DUMP
merged_piv.to_excel(writer, sheet_name='Sheet5-ISP+GCEX Sum')
writer.save()
####### CHANGE NAME HERE
# merged_piv.to_csv('C:\\Trade Vols\\202111 ISP+GCEX 1-15.csv')
